## Keeping long-term memories about collaboration 

![Short-term Memory vs. Long-term Memory](images/short-term%20memory%20vs%20long-term%20memory.png)

## Semantic Memory

![Semantic Memory](images/semantic-memory.png)

# Unstructured Memory

![Unstructured Memory](images/unstructured-memory.png)


# Define a proper User Profile schema

In [ ]:
from typing import List, Optional
from pydantic import BaseModel, Field

class UserProfile(BaseModel):
    name: str = Field(
        description="The user's preferred name.",
    )
    profession: str = Field(
        description="The user's profession or job title.",
    )
    seniority: Optional[str] = Field(
        default=None,
        description="The user's seniority level, e.g., 'mid-level', 'senior', etc. If unknown, guess it based on profession description",
    )
    languages: List[str] = Field(
        description="A list of programming languages the user is proficient in.",
    )
    frameworks: List[str] = Field(
        description="A list of frameworks or major technologies the user uses.",
    )
    current_project: Optional[str] = Field(
        default=None,
        description="The user's current main project.",
    )
    skills: List[str] = Field(
        description="A list of the user's technical or professional skills.",
    )
    current_interest: Optional[str] = Field(
        default=None,
        description="The user's current main interest or focus, e.g., a topic or technology they are actively exploring or asking about.",
    )

# Structured Outputs

In [ ]:
import json
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-4o-mini")
structured_model = model.with_structured_output(UserProfile)

profile = structured_model.invoke([HumanMessage(content="""
Hey, how are you? I am Evgeny, I am a software engineer and I need your help with 
my project. This is a TaskManager Java Spring Boot application. I do not get how
to configure security there!
""")])
print(profile.model_dump_json(indent=4))

# Structure Output

![Structure Output](images/structured-output.png)

# Let's recreate a simple profile in memory

In [ ]:
from langgraph.store.memory import InMemoryStore

long_term_memory = InMemoryStore()

user_id = "1"
namespace = (user_id, "memory")
key = "profile"

profile = {
  "name": "Evgeny"
}

long_term_memory.put(namespace, key, profile)

## Chat bot with structured profile

In [ ]:
from IPython.display import Image, display
from langchain_openai import ChatOpenAI

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.runnables.config import RunnableConfig
from langgraph.store.base import BaseStore
from langchain_core.messages import HumanMessage, SystemMessage

model = ChatOpenAI(model="gpt-4o-mini")

### Nodes

def chat(state: MessagesState, config: RunnableConfig, store: BaseStore):
    user_id = config["configurable"]["user_id"]

    # Retrieve memory from the store
    profile = store.get((user_id, "memory"), "profile")

    # Extract the actual memory content if it exists and add a prefix
    if profile:
        profile_content = profile.value
    else:
        profile_content = None

    # Format the memory in the system prompt
    system_msg = f"""
    You are a helpful assistant with memory capabilities.
    If user-specific memory is available, use it to personalize 
    your responses based on what you know about the user.
    
    Your goal is to provide relevant, friendly, and tailored 
    assistance that reflects the user’s preferences, context, and past interactions.

    If the user’s name or relevant personal context is available, always personalize your responses by:
        – Addressing the user by name (e.g., "Sure, Bob...") when appropriate
        – Referencing known projects, tools, or preferences (e.g., "your MCP  server typescript based project")
        – Adjusting the tone to feel friendly, natural, and directly aimed at the user

    Avoid generic phrasing when personalization is possible. For example, instead of "In TypeScript apps..." say "Since your project is built with TypeScript..."

    Use personalization especially in:
        – Greetings and transitions
        – Help or guidance tailored to tools and frameworks the user uses
        – Follow-up messages that continue from past context

    Always ensure that personalization is based only on known user details and not assumed.
    Tailor your answers to level of seniority of the user if it's known.
    
    The user’s profile (which may be empty) is provided as JSON (extract all data and use it): {profile_content}
    """

    response = model.invoke([SystemMessage(content=system_msg)] + state["messages"])

    return {"messages": response}


def update_memory(state: MessagesState, config: RunnableConfig, store: BaseStore):
    user_id = config["configurable"]["user_id"]
    namespace = (user_id, "memory")
    key = "profile"
    profile = store.get(namespace, key)
        
    if profile:
        profile_content = profile.value
    else:
        profile_content = None


    # Format the memory in the system prompt
    system_msg = f"""
    Update the user profile using the user's chat history.
    Save this for future reference. If a profile already exists, just update it.
    Here is the current profile (it might be empty): {profile_content}
    """
    
    updated_profile = structured_model.invoke([SystemMessage(content=system_msg)] + state['messages'])
    store.put(namespace, key, updated_profile.model_dump())


# Define the graph
builder = StateGraph(MessagesState)
builder.add_node("chat", chat)
builder.add_node("update_memory", update_memory)
builder.add_edge(START, "chat")
builder.add_edge("chat", "update_memory")
builder.add_edge("update_memory", END)

short_term_memory = MemorySaver()

graph = builder.compile(checkpointer=short_term_memory, store=long_term_memory)

display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

In [ ]:
memory = long_term_memory.get(namespace, key)
memory.dict()

In [ ]:
# start a new conversation
thread = {"configurable": {"thread_id": "1", "user_id": "1"}}

# define intiial user request
initial_input = {"messages": HumanMessage(content="""
Hey, I am a junior software engineer and I need your help with my project. 
This is a TaskManager Java Spring Boot application.
""")}

# run the graph and stream in values mode
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

In [ ]:
user_id = thread["configurable"]["user_id"]
namespace = (user_id, "memory")
key = "profile"
profile = long_term_memory.get(namespace, key)
profile.dict()

In [ ]:
# start a new conversation
thread = {"configurable": {"thread_id": "2", "user_id": "1"}}

# define intiial user request
initial_input = {"messages": HumanMessage(content="""
I do not get how to set up security configuration!
""")}

# run the graph and stream in values mode
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

In [ ]:
user_id = thread["configurable"]["user_id"]
namespace = (user_id, "memory")
key = "profile"
profile = long_term_memory.get(namespace, key)
profile.dict()